In [14]:
import numpy as np
from scipy.stats import uniform, expon, norm
from numpy import random

from ipywidgets import *
import ipywidgets as widgets
from ipysheet import from_array, to_array, sheet, cell

In [15]:
random.seed()

In [16]:
T = widgets.IntSlider(
    min=0,
    max=10000,
    step=50,
    value = 50,
    description='Время:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(T)


IntSlider(value=50, continuous_update=False, description='Время:', max=10000, step=50)

In [17]:
N = widgets.IntSlider(
    min=0,
    max=2000,
    step=1,
    value = 3,
    description='Размер:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(N)


IntSlider(value=3, continuous_update=False, description='Размер:', max=2000)

In [18]:
P = widgets.FloatSlider(min=0.0, max=1.0, step=0.05, value = 0.3)

display(P)

FloatSlider(value=0.3, max=1.0, step=0.05)

In [19]:
delta = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value = 0.01)
display(delta)

FloatSlider(value=0.01, max=1.0, step=0.01)

In [20]:
dt = delta.value

In [21]:
#распределение генератора заявок

A = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 1.0)
B = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 3.0)

display(A, B)
a = A.value 
b = B.value 

if (b < a):
    b, a = a, b


FloatSlider(value=1.0, max=10.0, step=0.5)

FloatSlider(value=3.0, max=10.0, step=0.5)

In [22]:
#распределение ОА
L = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 4)
display(L)

FloatSlider(value=4.0, max=10.0, step=0.5)

In [23]:
l = L.value

In [24]:
#следующая заявка
def next_request():
    return random.uniform(a, b)

#следующее время обработки 
def next_time():
    return random.exponential(l)

In [25]:
#метод дельта t
_rnum = 0 #количество обработанных заявок
_rnotnum = 0 #количество необработанных заявок
_rretnum = 0 #количество возвращенных заявок

_delta = next_request() #когда поступит заявка?
_time = _delta + next_time() #когда заявка обработается?

_runtime = 0 #время моделирования

optimal = 0
max_optimal = 0

que = 0
flag = False
while _runtime <= T.value: #процесс моделирования
    #print("que", que)
    if _delta <= _runtime: #генерируется заявка
        #можем ли мы принять эту заявку?
        if flag: #не можем
            optimal += 1 #считаем оптимальную очередь
            max_optimal = max(optimal, max_optimal)
            if (que + 1 <= N.value):
                que += 1 #положили заявку
            else:
                _rnotnum += 1 #не обработали заявку
        else: #можем
            flag = True
            _time = _delta + next_time()
        _delta += next_request() #протягиваем во времени
        
    if _time <= _runtime: #обрабатываем заявку
        flag = False
        _rnum += 1
       #отправится ли заявка назад?
        if (random.random() <= P.value):
            _rretnum += 1 #да
            flag = True
            _time = _time + next_time()
        else:
            if (optimal > 0):
                optimal -= 1 
            if (que > 0):
                que -= 1 
                flag = True
                _time = _time + next_time()
            elif (que == 0):
                _time = _delta + next_time()
    
    _runtime += dt #обновили время
         
            
            
        

In [26]:
s1 = sheet(6, 1, column_headers = list(['Всего заявок', 'Обработано', 'Не обработано', 'Возвращено', 'Осталось','Оптмальный размер очереди']))
cell(0, 0, _rnum+ _rnotnum)            # The Cell type will be 'numeric'
cell(0, 1,_rnum)           # The Cell type will be 'checkbox'
cell(0, 2,_rnotnum)
cell(0, 3,_rretnum)
cell(0, 4,max_optimal)
cell(0, 5, max_optimal)
s1

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='numeric', value=23), Cell(column…

In [15]:
#метод событий
_rnum = 0 #количество обработанных заявок
_rnotnum = 0 #количество необработанных заявок
_rretnum = 0 #количество возвращенных заявок

_delta = next_request() #когда поступит заявка?
_time = _delta + next_time() #когда заявка обработается?

_runtime = 0 #время моделирования

optimal = 0
max_optimal = 0

que = 0
flag = False
while _runtime <= T.value: #процесс моделирования
    #print("que", que)
    if _delta < _time: #что раньше? генерация или обработка
        #генерация
        if flag: #ОА занят
            optimal += 1 #считаем оптимальную очередь
            max_optimal = max(optimal, max_optimal)
            if (que + 1 <= N.value):
                que += 1 #положили заявку в очередь
            else:
                _rnotnum += 1 #не обработали заявку
        else: #ОА свободен
            flag = True #заняли ОА
            _time = _delta + next_time() 
        _time -= _delta
        _runtime += _delta
        _delta = next_request() 
        
    if _time <= _delta: #обрабатываем заявку
        flag = False
        _rnum += 1
        
        _delta -= _time
        _runtime += _time
       #отправится ли заявка назад?
        if (random.random() <= P.value):
            _rretnum += 1 #да
            flag = True
            _time = next_time()
        else:
            if (optimal > 0):
                optimal -= 1 
            if (que > 0):
                que -= 1 
                flag = True
                _time = next_time()
            elif (que == 0):
                _time = _delta + next_time()        

In [16]:
s1 = sheet(6, 1, column_headers = list(['Всего заявок', 'Обработано', 'Не обработано', 'Возвращено', 'Осталось','Оптмальный размер очереди']))
cell(0, 0, _rnum+ _rnotnum)            # The Cell type will be 'numeric'
cell(0, 1,_rnum)           # The Cell type will be 'checkbox'
cell(0, 2,_rnotnum)
cell(0, 3,_rretnum)
cell(0, 4,max_optimal)
cell(0, 5, max_optimal)
s1

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='numeric', value=25), Cell(column…